<a href="https://colab.research.google.com/github/Qukoyk/oddball/blob/master/LuSync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

哈喽   

这个是一个比较粗糙而丑陋的示例。  
具体哪里丑陋，接下来会说。    
本地的Python环境好像有点问题，我在Google Colab里写的。你可以理解为是一个在线编辑环境。  
所以4-9行的写法，如果本地载入的话不需要这么写。   


---

首先第一部分是加载库和数据。  
这边加载了pandas，还有Google Colab需要用到的东西。  
指定好文件路径后，读取文件。需要注意的是，FEI.xlsx里面的第一行是个只有一列的小title，需要跳过去，所以会有第12行的  


```
……, skiprows=1)
```
读取完了之后，就可往下走。为了展示，我添加了15-16行和下面的展示。


In [43]:
import pandas as pd
from google.colab import drive

# 加载Google Drive
drive.mount('/content/drive')
# 指定文件路径
path = "/content/drive/My Drive/Lu/"
fileMarkerName = 'FEI_Marker(1).xlsx'
fileName = 'FEI.xlsx'

# 读取文件
df = pd.read_excel(path+fileName, skiprows=1)
dfMarker = pd.read_excel(path+fileMarkerName)

# 展示文件
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Timestamp,Unnamed: 1,Marker,POW.AF3.Theta,POW.AF3.Alpha,POW.AF3.BetaL,POW.AF3.BetaH,POW.AF3.Gamma,POW.F7.Theta,POW.F7.Alpha,...,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma
0,1.676271e+09,00:00:00.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.676271e+09,00:00:00.004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.676271e+09,00:00:00.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.676271e+09,00:00:00.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.676271e+09,00:00:00.016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75743,1.676272e+09,00:04:55.713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75744,1.676272e+09,00:04:55.717,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75745,1.676272e+09,00:04:55.721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75746,1.676272e+09,00:04:55.725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


下面这一段的主要作用就是找Trigger点，然后下面增加100个。   

好了，丑陋的地方来了——   
pandas其实能在内部执行条件判断，像if那样的东西。但是我没有好好学，直接转换成了数组，用熟悉的数组操作搞的。   
相当于绕了一圈。你学Pandas的时候可以好好看看正确的用法，不用像我这样。

In [56]:
# 把数据中按照Marker那一列，查找值不为Nan的，从DataFrame转成数组
markerIndexList = df[df['Marker'].notna()].index.tolist()
markerList = df[df['Marker'].notna()].values.tolist()

# 建立空数组
# 保存未来Marker行号的数组
hundredList = []
# 保存未来Marker内容的数组
hundredMarkerList = []
# 各个Marker行号的数组
snList = []
scList = []
nnList = []
ncList = []
# 计数器
counter = 0

# 遍历现有Marker行号
for i in markerIndexList:
    # 通过另一个遍历，往下添加100行
    for j in range(100):
        # 找到Marker内容
        markerValue = markerList[counter][2]
        # 添加进相应数组
        hundredList.append(i+j)
        hundredMarkerList.append(markerValue)
        # 把不同的Marker添加进相应数组
        if markerValue == 'SN':
            snList.append(i+j)
            pass
        elif markerValue == 'SC':
            scList.append(i+j)
            pass
        elif markerValue == 'NN':
            nnList.append(i+j)
            pass
        elif markerValue == 'NC':
            ncList.append(i+j)
            pass
        pass
    # 计数器累进
    counter = counter + 1
    pass

# 大概预览一下，看对不对
print(scList)
print(hundredList)
print(hundredMarkerList)

[28311, 28312, 28313, 28314, 28315, 28316, 28317, 28318, 28319, 28320, 28321, 28322, 28323, 28324, 28325, 28326, 28327, 28328, 28329, 28330, 28331, 28332, 28333, 28334, 28335, 28336, 28337, 28338, 28339, 28340, 28341, 28342, 28343, 28344, 28345, 28346, 28347, 28348, 28349, 28350, 28351, 28352, 28353, 28354, 28355, 28356, 28357, 28358, 28359, 28360, 28361, 28362, 28363, 28364, 28365, 28366, 28367, 28368, 28369, 28370, 28371, 28372, 28373, 28374, 28375, 28376, 28377, 28378, 28379, 28380, 28381, 28382, 28383, 28384, 28385, 28386, 28387, 28388, 28389, 28390, 28391, 28392, 28393, 28394, 28395, 28396, 28397, 28398, 28399, 28400, 28401, 28402, 28403, 28404, 28405, 28406, 28407, 28408, 28409, 28410, 31399, 31400, 31401, 31402, 31403, 31404, 31405, 31406, 31407, 31408, 31409, 31410, 31411, 31412, 31413, 31414, 31415, 31416, 31417, 31418, 31419, 31420, 31421, 31422, 31423, 31424, 31425, 31426, 31427, 31428, 31429, 31430, 31431, 31432, 31433, 31434, 31435, 31436, 31437, 31438, 31439, 31440, 31441

好了，现在我们有了各个Marker要插入的行号了，现在就可插入了。    
以防万一，这边对原有的DataFrame进行复制，在副本中进行操作。

In [57]:
# 制作DataFrame副本
g = df.copy()
# 把各个Marker，按照行号，写入副本相应位置
g.loc[scList,['Marker']] = 'SC'
g.loc[snList,['Marker']] = 'SN'
g.loc[ncList,['Marker']] = 'NC'
g.loc[nnList,['Marker']] = 'NN'
# 展示看下对不对
g.loc[hundredList,['Marker']]

,Marker
19433,SN
19434,SN
19435,SN
19436,SN
19437,SN
...,...
73590,NN
73591,NN
73592,NN
73593,NN


没什么问题，通过下方的后缀，去掉Pandas自带的行号，保存文件。


```
……, index=None)
```



In [61]:
g.to_excel('with100Marker_NoIndex.xlsx', index=None)

好了，这就是一个简单快速但有些丑陋的扩写Trigger的样例。   
你可以在[这里](https://colab.research.google.com/github/Qukoyk/oddball/blob/master/CV_pr.ipynb)看到博一时写的更丑陋的（笑   
希望能帮到你  
   
Life is short, use Python.

Qukoyk